# NDVI Generation using VirtuGhan
https://github.com/kshitijrajsharma/VirtuGhan 

In [1]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import leafmap

In [2]:
import os, shutil

output_dir = "virtughan_output_cloud_10"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [3]:
import geopandas as gpd

geemap.ee_initialize()geemap.ee_initialize()
aoi_path = "../data/aoi_boundary.geojson"
aoi = gpd.read_file(aoi_path)
aoi = aoi.to_crs("EPSG:4326")  # ensure it's in lat/lon

bbox = list(aoi.total_bounds)
print("Extracted BBOX:", bbox)

Extracted BBOX: [np.float64(15.192582250690464), np.float64(-4.389991985869003), np.float64(15.405144373610312), np.float64(-4.23702926871141)]


In [4]:
from vcube.engine import VCubeProcessor

# Setup parameters
start_date = "2014-01-01"
end_date = "2025-01-01"
cloud_cover = 10
formula = "(band2-band1)/(band2+band1)"  # NDVI
band1 = "red"
band2 = "nir"
operation = "median"
timeseries = True
workers = 10
smart_filter = False

processor = VCubeProcessor(
    bbox,
    start_date,
    end_date,
    cloud_cover,
    formula,
    band1,
    band2,
    operation,
    timeseries,
    output_dir,
    workers=workers,
    smart_filter=smart_filter
)

processor.compute()


Engine starting...
Searching STAC .....
Total scenes found: 103
Scenes covering input area: 48
Scenes after removing overlaps: 37
Using Parallel Processing...
Computing Band Calculation: 100%|██████████| 37/37 [50:44<00:00, 82.30s/it]   
Aggregating results...
Saving aggregated result with colormap...
Creating GIF and zipping TIFF files...
Saved timeseries GIF to virtughan_output_cloud_10/output.gif
Saved intermediate images ZIP to virtughan_output_cloud_10/tiff_files.zip


In [5]:
import glob
import imageio

# Get the output images
image_files = sorted(glob.glob(f"{output_dir}/*.png"))

# Create GIF
gif_path = "aoi_ndvi_timelapse.gif"
with imageio.get_writer(gif_path, mode="I", duration=0.5) as writer:
    for filename in image_files:
        image = imageio.imread(filename)
        writer.append_data(image)

print(f"✅ Timelapse created: {gif_path}")


✅ Timelapse created: aoi_ndvi_timelapse.gif
